# Example use of the Sonic module Evo.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [1]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [2]:
import Dates, JuliaDB, Random, StatsPlots

In [3]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

Plots.GRBackend()

Get secret token,

In [4]:
include("../src/EvoSecrets.jl")
secret = EvoSecrets.veab

(source = (endpoint = "https://evo.elvaco.se/api/v1", username = "jens.brage@noda.se", password = "bV6CDHnC&tMm"), target = "db/evo/veab")

Get access token,

In [60]:
include("../src/Evo.jl")
access = Evo.get_access(secret)

(source = (endpoint = "https://evo.elvaco.se/api/v1", username = "jens.brage@noda.se", password = "bV6CDHnC&tMm"), target = "db/evo/veab", bearer = "85f790f7-5f37-4695-8cbe-129b5ae3ff31")

`GET https://evo.elvaco.se/api/v1/meters`,

In [75]:
dates = Dates.DateTime(2017, 12), Dates.DateTime(2019, 12)

(2017-12-01T00:00:00, 2019-12-01T00:00:00)

In [49]:
@time meters = Evo.get_meters(access, dates..., 0)

  1.230238 seconds (27.56 k allocations: 1.581 MiB)


Dict{String,Any} with 11 entries:
  "number"           => 0
  "empty"            => false
  "pageable"         => Dict{String,Any}("pageSize"=>50,"unpaged"=>false,"sort"…
  "totalPages"       => 105
  "last"             => false
  "content"          => Any[Dict{String,Any}("gatewaySerial"=>"12133603","addre…
  "totalElements"    => 5221
  "first"            => true
  "size"             => 50
  "sort"             => Dict{String,Any}("unsorted"=>true,"sorted"=>false,"empt…
  "numberOfElements" => 50

Get content,

In [50]:
@time content = Evo.get_content(access, dates...)
length(content)

 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105,126.930533 seconds (687.77 k allocations: 49.500 MiB, 0.03% gc time)


5221

In [54]:
t = JuliaDB.table(content; pkey = :id)

Table with 5221 rows, 10 columns:
Columns:
#   colname               type
──────────────────────────────────────────
1   collectionPercentage  Float64
2   facility              Symbol
3   gatewaySerial         Symbol
4   id                    Symbol
5   isReported            Bool
6   location              Dict{String,Any}
7   manufacturer          Symbol
8   medium                Symbol
9   organisationId        Symbol
10  readIntervalMinutes   Int64

In [57]:
site = (content = t, )
map(length, site)

(content = 5221,)

Save site,

In [58]:
@time Evo.savesite(secret; site...)

 13.297115 seconds (11.51 M allocations: 716.593 MiB, 2.11% gc time)


Load site,

In [61]:
@time site = Evo.loadsite(secret, :content)
map(length, site)

  0.157914 seconds (270.38 k allocations: 13.444 MiB)


(content = 5221,)

Specify patterns,

In [68]:
pattern = Dict(
    Symbol("Energy")                => identity,
    Symbol("Flow")                  => identity,
    Symbol("Forward temperature")   => identity,
    Symbol("Power")                 => identity,
    Symbol("Return temperature")    => identity,
    Symbol("Volume")                => identity,
)

Dict{Symbol,typeof(identity)} with 6 entries:
  :Energy                       => identity
  :Volume                       => identity
  Symbol("Return temperature")  => identity
  :Flow                         => identity
  Symbol("Forward temperature") => identity
  :Power                        => identity

Get pattern nodes,

In [71]:
rs = site.content # JuliaDB.rows(Evo.get_pattern_nodes(patterns; site...))
if true
    rs = [(id = Symbol("0016a5f0-ff31-4dda-9465-0aaf841ce2dd"), )] # [rs[Random.rand(1 : end)]]
end
length(rs)

1

Save data,

In [76]:
@time for r in rs
    for sensor_name in keys(pattern)
        p = secret.target
        p = Evo.dbpath(p, "data", r.id; db = sensor_name)
        if !isfile(p)
            try
                Evo.savedata(access, r.id, sensor_name, dates)
                println("$(p): Done")
            catch e
                if typeof(e) <: InterruptException
                    rethrow(e)
                end
                println("$(p): $(e)")
            end
        end
    end
end

db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Energy.db: ErrorException("type NamedTuple has no field endpoint")
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Volume.db: ErrorException("type NamedTuple has no field endpoint")
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Return temperature.db: ErrorException("type NamedTuple has no field endpoint")
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Flow.db: ErrorException("type NamedTuple has no field endpoint")
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Forward temperature.db: ErrorException("type NamedTuple has no field endpoint")
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Power.db: ErrorException("type NamedTuple has no field endpoint")
  1.213829 seconds (2.41 M allocations: 126.966 MiB, 4.87% gc time)


Random choice,

In [ ]:
r = rs[Random.rand(1 : end)]

In [ ]:
sensor_names = collect(keys(patterns[r.device_id]))

In [ ]:
sensor_name = sensor_names[Random.rand(1 : end)]

Load data,

In [ ]:
data = Linckii.loaddata(secret, r.node_id, sensor_name)

Simple graphics,

In [ ]:
@df data plot(:datetime, :value, label = Linckii.dbpath(r.node_id; db = sensor_name))